# Modeling the focal plane of a large-format groundbased astronomical imaging instrument

This notebook shows how to fit data from a large-format camera with a parsimonious model that accounts for the optical distortion terms and chip gaps.

**Author**: David Shupe, Caltech/IPAC

Refer to: SciPy 2018 talk

Here are the steps:

1. Retrieve a set of PSF-fit catalogs from public ZTF data, for a single exposure.
2. Combine these catalogs into a single table with five columns for RA & Dec, local x-pixel and y-pixel, and chip number.
3. Use the statsmodels package to fit a model with terms for chip gaps, small rotations between chips, and overall optical distortion.
4. Write out a `.ahead` file for use with SCAMP from the Astromatic suite.

## Imports (all here to make sure we have them)

In [ ]:
import numpy as np
from astropy.io import fits
from astropy.table import Table
from astropy.wcs import WCS
from astropy.utils.data import download_file
import pandas as pd
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
import seaborn as sns

## Download catalogs for a single science exposure for an extragalactic field

We need a ZTF exposure that is public and that contains data for all 64 quadrants.

Data meeting these requirements are available at https://irsa.ipac.caltech.edu/ibe/data/ztf/products/sci/2019/0408/164213/

In [ ]:
template_url = ('https://irsa.ipac.caltech.edu/ibe/data/ztf/products/sci/2019/0408' +
                 '/164213/ztf_20190408164213_000747_zr_c01_o_q1_psfcat.fits')

Download the image corresponding to the template catalog and extract some metadata

In [ ]:
sci_header = fits.getheader(template_url.replace('_psfcat.fits', '_sciimg.fits'))

In [ ]:
sci_header.get('TELRAD')

In [ ]:
sci_header.get('TELDECD')

Make a WCS with a projection center at  crval1=TELRAD, crval2=TELDECD, with cdelt1 = cdelt2 = 1.0 degrees and a TAN projection.

In [ ]:
proj_wcs = WCS(naxis=2)
proj_wcs.wcs.ctype = ['RA---TAN', 'DEC--TAN']
proj_wcs.wcs.crval = [sci_header.get('TELRAD'), sci_header.get('TELDECD')]
proj_wcs.wcs.cdelt = [1.0, 1.0] # one degree per "pixel"
proj_wcs.wcs.crpix = [4.5, 4.5]
proj_wcs.array_shape = [8, 8] # NAXIS2, NAXIS1

Use the template catalog to download and get columns from all the PSF catalogs.

In [ ]:
psfcat = Table.read(download_file(template_url), format='fits')

In [ ]:
psfcat.columns

Get the data for all 64 quadrants aka readout channels

In [ ]:
xvals = []
yvals = []
etavals = []
nuvals = []
rcids = []
ras = []
decs = []

for ccd in range(1, 17):
    for quadrant in range(1, 5):
        psf_url = template_url.replace('_c01_o_q1_psfcat.fits', f'_c{ccd:02}_o_q{quadrant:01}_psfcat.fits')
        fname = download_file(psf_url, cache=True)
        header = fits.getheader(fname)
        tab = Table.read(fname, format='fits')
        plane_coords = proj_wcs.wcs_world2pix(np.vstack([tab['ra'], tab['dec']]).T, 1)
        rcids.append(header['rcid']*np.ones_like(tab['ra']))
        xvals.append(tab['xpos'])
        yvals.append(tab['ypos'])
        etavals.append(plane_coords[:,0])
        nuvals.append(plane_coords[:,1])
        ras.append(tab['ra'])
        decs.append(tab['dec'])


In [ ]:
x_image = np.hstack(xvals)
y_image = np.hstack(yvals)
rcid = np.array(np.hstack(rcids), dtype=np.int32)
eta = np.hstack(etavals)
nu = np.hstack(nuvals)
ra = np.hstack(ras)
dec = np.hstack(decs)

The table we need is five columns and contains the matched stars

In [ ]:
df = pd.DataFrame({'rcid':rcid, 'x_image':x_image, 'y_image':y_image,
                   'eta':eta, 'nu':nu, 'ra':ra, 'dec':dec})

Note that `x_image` and `y_image` are in the FITS convention and are local to each quadrant

In [ ]:
df.head()

How many stars do we have?

In [ ]:
len(df)

At this point one could make a cut in magnitude...

## Fit a linear model

We first fit a linear model so that we can get first-order chip gaps that will allow us to define a global pixel coordinate.

Do the "eta" direction first (aligned with Right Ascension).

In [ ]:
xmod = smf.ols(formula='eta ~ x_image + y_image + C(rcid) -1 + ' +
               'C(rcid)*x_image + C(rcid)*y_image ', data=df)

Fit.

In [ ]:
xres = xmod.fit()

In [ ]:
print(xres.summary())

In [ ]:
print(xres.summary())

Fit the "nu" direction which is aligned with Declination.

In [ ]:
ymod = smf.ols(formula='nu ~ x_image + y_image + C(rcid) -1 + ' +
               'C(rcid)*x_image + C(rcid)*y_image ', data=df)

In [ ]:
yres = ymod.fit()

In [ ]:
print(yres.summary())

What is our "global CD matrix"?

In [ ]:
print(xres.params.get('x_image'), xres.params.get('y_image'))
print(yres.params.get('x_image'), yres.params.get('y_image'))


Make a dictionary of the parameters for the linear fit.

In [ ]:
rcmeta = {}
for i in range(64):
    cd11 = xres.params.get('x_image')
    cd12 = xres.params.get('y_image')
    cd21 = yres.params.get('x_image')
    cd22 = yres.params.get('y_image')
    if i > 0:
        cd11 += xres.params.get('C(rcid)[T.{}]:x_image'.format(i))
        cd12 += xres.params.get('C(rcid)[T.{}]:y_image'.format(i))
        cd21 += yres.params.get('C(rcid)[T.{}]:x_image'.format(i))
        cd22 += yres.params.get('C(rcid)[T.{}]:y_image'.format(i))
    cd = np.matrix([[cd11, cd12], [cd21, cd22]])
    invcd = cd**-1
    offset_degrees = np.matrix([[-xres.params.get('C(rcid)[{}]'.format(i))],
                           [-yres.params.get('C(rcid)[{}]'.format(i))]])
    offset_pix = invcd*offset_degrees
    rcmeta[i] = dict(cd11=float(cd11), cd12=float(cd12), cd21=float(cd21), cd22=float(cd22),
                     crpix1=float(offset_pix.item(0)), crpix2=float(offset_pix.item(1)))
    

What do our residuals look like? Express these as arcseconds.

In [ ]:
df['etaresid'] = xres.resid*3600
df['nuresid'] = yres.resid*3600

Plot the x-residuals for all 64 quadrants.

In [ ]:
%matplotlib inline
gx1 = sns.FacetGrid(df, col="rcid", col_wrap=4, 
                    height=6, aspect=1)
gx1.map(plt.scatter, "x_image", "etaresid", color="#334488", edgecolor="white", lw=.5)
gx1.set(ylim=(-5,5))

Plot the y-residuals for all 64 quadrants.

In [ ]:
%matplotlib inline
gy1 = sns.FacetGrid(df, col="rcid", col_wrap=4, 
                    height=6, aspect=1)
gy1.map(plt.scatter, "y_image", "nuresid", color="#334488", edgecolor="white", lw=.5)
gy1.set(ylim=(-5,5))

## Form global pixel coordinates and fit a quadratic model

In [ ]:
df['x_global'] = df['x_image']
df['y_global'] = df['y_image']
df['etalin'] = df['eta']
df['nulin'] = df['nu']

In [ ]:
for i in  range(64):
    df.loc[df.rcid==i, 'x_global'] = df.loc[df.rcid==i, 'x_image'] - rcmeta[i]['crpix1']
    df.loc[df.rcid==i, 'y_global'] = df.loc[df.rcid==i, 'y_image'] - rcmeta[i]['crpix2']
    df.loc[df.rcid==i, 'etalin'] = (df.loc[df.rcid==i, 'x_global']*rcmeta[i]['cd11'] +
                                    df.loc[df.rcid==i, 'y_global']*rcmeta[i]['cd12'])
    df.loc[df.rcid==i, 'nulin'] = (df.loc[df.rcid==i, 'x_global']*rcmeta[i]['cd21'] +
                                    df.loc[df.rcid==i, 'y_global']*rcmeta[i]['cd22'])

In [ ]:
df.head()

Now do a fit to our new "global pixel coordinates" that is quadratic to account for the optical distortion.

In [ ]:
xmod = smf.ols(formula='eta ~ np.power(etalin,2) + np.power(nulin,2) + ' + 
               'etalin*nulin + etalin + nulin' , data=df)

In [ ]:
xres = xmod.fit()

In [ ]:
print(xres.summary())

In [ ]:
ymod = smf.ols(formula='nu ~ np.power(etalin,2) + np.power(nulin,2) + ' + 
               'etalin*nulin + etalin + nulin', data=df)

In [ ]:
yres = ymod.fit()

In [ ]:
print(yres.summary())



What are the residuals now?

In [ ]:
df['etaresid2'] = xres.resid*3600
df['nuresid2'] = yres.resid*3600

In [ ]:
%matplotlib inline
gx1 = sns.FacetGrid(df, col="rcid", col_wrap=4, 
                    height=6, aspect=1)
gx1.map(plt.scatter, "x_image", "etaresid2", color="#334488", edgecolor="white", lw=.5)
gx1.set(ylim=(-5,5))

In [ ]:
%matplotlib inline
gy1 = sns.FacetGrid(df, col="rcid", col_wrap=4, 
                    height=6, aspect=1)
gy1.map(plt.scatter, "y_image", "nuresid2", color="#334488", edgecolor="white", lw=.5)
gy1.set(ylim=(-5,5))

PV1_4 is x^2, PV1_5 is xy, PV1_6 is y^2

PV2_4 is y^2, PV2_5 is xy, PV2_6 is x^2

In [ ]:
pv_start = dict(PV1_1 = float(xres.params.get('etalin')),
               PV1_2 = float(xres.params.get('nulin')),
               PV1_4 = float(xres.params.get('np.power(etalin, 2)')),
               PV1_5 = float(xres.params.get('etalin:nulin')),
               PV1_6 = float(xres.params.get('np.power(nulin, 2)')),
               PV2_1 = float(yres.params.get('nulin')),
               PV2_2 = float(yres.params.get('etalin')),
               PV2_4 = float(yres.params.get('np.power(nulin, 2)')),
               PV2_5 = float(yres.params.get('etalin:nulin')),
               PV2_6 = float(yres.params.get('np.power(etalin, 2)'))
            )

Print out the TPV distortion coefficients we have derived for our prior.

In [ ]:
for k in sorted(pv_start.keys()):
    print('{}: {}'.format(k,pv_start[k]))

In [ ]:
switches = dict(reset_crpix = True, reset_cdmatrix = True)

Print the linear parameters that the telescope control system fills in

In [ ]:
for ccdid in range(1,17):
    for qid in [2,3,0,1]:
        k = (ccdid-1)*4 + qid
        print('WCSDATA="%2d  %d  %13.10f %13.10f %13.10f %13.10f %10.3f %10.3f 0.0"' %
             (ccdid, qid, rcmeta[k]['cd11'], rcmeta[k]['cd12'], rcmeta[k]['cd21'],
              rcmeta[k]['cd22'], rcmeta[k]['crpix1'], rcmeta[k]['crpix2']))

## Output a .ahead file for SCAMP

Pick a quadrant and show how to output a .ahead file

In [ ]:
my_rcid = 32

In [ ]:
output_header = fits.Header()

In [ ]:
output_header['NAXIS'] = 2
output_header['NAXIS1'] = 3072
output_header['NAXIS2'] = 3080
output_header['CTYPE1'] = 'RA---TPV'
output_header['CTYPE2'] = 'DEC--TPV'
output_header['CRVAL1'] = sci_header['TELRAD']
output_header['CRVAL2'] = sci_header['TELDECD']
output_header['CRPIX1'] = rcmeta[my_rcid]['crpix1']
output_header['CRPIX2'] = rcmeta[my_rcid]['crpix2']
output_header['CD1_1'] = rcmeta[my_rcid]['cd11']
output_header['CD1_2'] = rcmeta[my_rcid]['cd12']
output_header['CD2_1'] = rcmeta[my_rcid]['cd21']
output_header['CD2_2'] = rcmeta[my_rcid]['cd22']
output_header['PV1_1'] = pv_start['PV1_1']
output_header['PV1_2'] = pv_start['PV1_2']
output_header['PV1_4'] = pv_start['PV1_4']
output_header['PV1_5'] = pv_start['PV1_5']
output_header['PV1_6'] = pv_start['PV1_6']
output_header['PV2_1'] = pv_start['PV2_1']
output_header['PV2_2'] = pv_start['PV2_2']
output_header['PV2_4'] = pv_start['PV2_4']
output_header['PV2_5'] = pv_start['PV2_5']
output_header['PV2_6'] = pv_start['PV2_6']

Output it to a file

In [ ]:
output_header.totextfile(f'quadrant_{my_rcid:02}.ahead', overwrite=True)    

Our astrometry prior is ready to be used by SCAMP!